<a href="https://colab.research.google.com/github/Seeker220/drive-view-only-downloader/blob/main/GDrive_View_Only_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download View Only PDF:


## Install Dependencies

In [ ]:
!pip install img2pdf

## Follow these steps
1. Mount your Drive, copy the Destination Folder path and paste it below.
2. Open the publicly shared PDF in an Incognito tab (ensure it’s accessible by anyone with the link).
3. Open Developer Tools and go to the Network tab.
4. Zoom in on the first page of the PDF.
5. In the Network tab, look for URLs like:
`img?ck=drive&....&page=0&...&w=3200&...`
Select the URL with the highest value of `w`.
6. Right-click on the URL and select `Copy` > `Copy URL`.
7. Run the script, when asked paste the url, type number of pages, and filename. Enjoy!

In [ ]:
dest = "/content/drive/MyDrive" # @param {"type":"string","placeholder":"Destination Folder"}
import os
import shutil
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import img2pdf

# Create images directory
images_dir = "images"
os.makedirs(images_dir, exist_ok=True)

# Function to download an image
def download_image(url, filename):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
    return filename if response.status_code == 200 else None

# Get user inputs
first_url = input("Enter the first image URL: ")
total_pages = int(input("Enter the total number of pages: "))
pdf_filename = input("Enter the PDF file name (without .pdf): ") + ".pdf"
pdf_filename = dest+"/"+pdf_filename
# Parse the URL
parsed_url = urlparse(first_url)
query_params = parse_qs(parsed_url.query)

image_filenames = []

# Prepare URLs for all pages
download_tasks = []
for page in range(total_pages):
    query_params["page"] = [str(page)]
    new_query = urlencode(query_params, doseq=True)
    new_url = urlunparse(parsed_url._replace(query=new_query))
    filename = os.path.join(images_dir, f"page_{page}.webp")
    download_tasks.append((new_url, filename))

# Download images in parallel
print("\nDownloading images...\n")
with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust thread count as needed
    future_to_filename = {executor.submit(download_image, url, filename): filename for url, filename in download_tasks}

    with tqdm(total=total_pages, desc="Progress", unit="page", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} pages', colour='green') as pbar:
        for future in as_completed(future_to_filename):
            filename = future_to_filename[future]
            if future.result():
                image_filenames.append(filename)
            pbar.update(1)

# Sort filenames to maintain order
image_filenames.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))

# Convert images to PDF using img2pdf
print("\nMerging images into PDF...\n")
with open(pdf_filename, "wb") as f:
    f.write(img2pdf.convert(image_filenames))

# Cleanup: Delete images directory
shutil.rmtree(images_dir)

print(f"\n✅ PDF saved as {pdf_filename} (Images deleted) 🚀")


# Download View Only Video:

## Install Dependencies

In [ ]:
!pip install yt-dlp
!apt update && apt install -y aria2
!apt update && apt install -y ffmpeg

## Follow these steps
1. Mount your Drive, copy the Destination Folder path and paste it below.
2. Enter filename without extension.
3. Enter the drive view-only video link. Make sure it is publicly accessible and looks like `https://drive.google.com/file/d/..../view`

In [ ]:
dest = "/content/drive/MyDrive" # @param {"type":"string","placeholder":"Destination Folder"}
filename = "video" # @param {"type":"string","placeholder":"File Name without Extension"}
video_url = "" # @param {"type":"string","placeholder":"Enter Drive View Only Link"}
!yt-dlp --downloader aria2c -o "{dest}/{filename}.%(ext)s" "{video_url}"